# Course : Linear programming

<center><img src="https://algorist.com/images/figures/linear-programming-L.png" title="LP "/></center>

This course covers the modeling keys you will need so you can start using python for linear programming.  

# 1. Importing data

In [1]:
#define the network structure

plants = ['P1', 'P2']

warehouses = ['w1', 'w2']

markets = ['c1', 'c2', 'c3']

#define plant capacity
plants_capacity = {'P1':200000 ,'P2':50000}

#define markets' demand
market_demand = {'c1':50000 , 'c2':100000, 'c3':50000}


In [2]:
#define a list of costs of each transportation path
transport_plant_warehouse = [ # warehouses
    # w1 w2
    [0, 5], #P1   # Plants
    [4, 2]] #P2

# The cost data is made into a dictionary
#costs_plant_warehouse = makeDict([plants, warehouses], transport_plant_warehouse, 0)


In [3]:
#costs_plant_warehouse

In [4]:
#define  a list of costs of each transportation path
transport_warehouse_market = [ # markets
    #c1 c2 c3
    [3, 4, 5], #w1 # warehouses
    [2, 1, 2]  #w2 
] 

# The cost data is made into a dictionary
#costs_warehouse_market = makeDict([warehouses, markets], transport_warehouse_market, 0)

#costs_warehouse_market

# 2. Create the model and solve it

In [2]:
# install the solver
!pip install gurobipy

# Import gurobi modeler functions
from gurobipy import *
from gurobipy import Model
from gurobipy import GRB

ModuleNotFoundError: No module named 'gurobipy'

<div class="alert alert-block alert-info">
Exercise : Create a model and define your decision variable

In [6]:
# Creates the 'prob' variable to contain the problem data
prob = Model('allocationModel')

Restricted license - for non-production use only - expires 2025-11-24


In [7]:
# the decision variables are created plant->warehouse
allocationVar_plant_warehouse = prob.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="allocP_W")

In [8]:
# the decision variables are created warehouse->market
allocationVar_warehouse_market = prob.addVars(warehouses, markets, vtype=GRB.CONTINUOUS, name="allocW_M")

<div class="alert alert-block alert-info">
Exercise : Add your first constraint, supply maximum constraints, capacity restriction

In [9]:
# the supply maximum constraints are added to prob for each plant
# 用于计算每个生产厂向所有仓库分配的总数量。这个总数量被限制为不超过该生产厂的最大产能
prob.addConstrs(
    (quicksum(allocationVar_plant_warehouse[plant_id,warehouse_id] for warehouse_id in warehouses) <= plants_capacity[plant_id] for plant_id in plants),
    name="CapLimitConstr"
)

{'P1': <gurobi.Constr *Awaiting Model Update*>,
 'P2': <gurobi.Constr *Awaiting Model Update*>}

<div class="alert alert-block alert-info">
Exercise : add the constraint of the demand covering to the model

In [10]:
# the market demand covering constraints are added to prob for each market

prob.addConstrs(
    
    (
        quicksum(allocationVar_warehouse_market[warehouse_id,market_id] for warehouse_id in warehouses)
        == 
        market_demand[market_id] 

    for market_id in markets ),
    
    name = "DemCovering"
)

{'c1': <gurobi.Constr *Awaiting Model Update*>,
 'c2': <gurobi.Constr *Awaiting Model Update*>,
 'c3': <gurobi.Constr *Awaiting Model Update*>}

<div class="alert alert-block alert-info">
Exercise : Add the flow balance constraints to the model

In [11]:
# the flow balance contraints are added to prob for each warehouse: in warehouse = to out warehouse
prob.addConstrs(

    (
        quicksum(allocationVar_plant_warehouse[plant_id,warehouse_id] for plant_id in plants) 
        == 
        quicksum(allocationVar_warehouse_market[warehouse_id,market_id] for market_id in markets)
        
    for warehouse_id in warehouses),

    name = 'equilibrium'
)


{'w1': <gurobi.Constr *Awaiting Model Update*>,
 'w2': <gurobi.Constr *Awaiting Model Update*>}

<div class="alert alert-block alert-info">
Exercise : Define your objective function and add it to the model

In [12]:
# The objective function is added to prob first

## cost plant -> warehouse
cost_plant_warehouse = quicksum(allocationVar_plant_warehouse[plant_id,warehouse_id] * transport_plant_warehouse[plants.index(plant_id)][warehouses.index(warehouse_id)] 
                                for plant_id in plants for warehouse_id in warehouses
                            )

## cost warehouse -> market
cost_warehouse_market = quicksum(allocationVar_warehouse_market[warehouse_id,market_id] * transport_warehouse_market[warehouses.index(warehouse_id)][markets.index(market_id)]
                                 for warehouse_id in warehouses for market_id in markets
)

## compute total cost
total_cost = cost_plant_warehouse + cost_warehouse_market


# add objective fucntion to prob var
prob.setObjective(total_cost,GRB.MINIMIZE)

<div class="alert alert-block alert-info">
Exercise : save the model into an lp form

In [13]:
# The problem data is written to an .lp file
prob.write("allocationExample.lp")

<div class="alert alert-block alert-info">
Exercise : Solve your model

In [14]:
# The problem is solved using gurobi's choice of Solver
prob.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0xc3d27b8c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+04, 2e+05]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+05   3.125000e+04   0.000000e+00      0s
       2    7.5000000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  7.500000000e+05


<div class="alert alert-block alert-info">
Exercise : Print out your decision variable values

In [15]:
allocationVar_warehouse_market['w1','c1'].X

50000.0

In [16]:
for v in prob.getVars():
    print(f"{v.Varname} = {v.X}")

allocP_W[P1,w1] = 150000.0
allocP_W[P1,w2] = 0.0
allocP_W[P2,w1] = 0.0
allocP_W[P2,w2] = 50000.0
allocW_M[w1,c1] = 50000.0
allocW_M[w1,c2] = 100000.0
allocW_M[w1,c3] = 0.0
allocW_M[w2,c1] = 0.0
allocW_M[w2,c2] = 0.0
allocW_M[w2,c3] = 50000.0
